# **IMPORT LIBRERIE**

In [1]:
import dlib
import csv
import cv2 as cv
import io
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(23)
import os
os.environ['PYTHONHASHSEED'] = '5'
import random
random.seed(666)
import tensorflow as tf
tf.random.set_seed(666)
import tensorflow_datasets as tfds


# **CLASSE FACE DETECTOR**

In [2]:
modelFile = "/home/paolo/ComputerSync/Università/ProgettoAgeEstimation/AgeEstimation/data/res10_300x300_ssd_iter_140000_fp16.caffemodel"
configFile = "/home/paolo/ComputerSync/Università/ProgettoAgeEstimation/AgeEstimation/data/deploy.prototxt"

class FaceDetector:
    net = None
    def __init__(self, min_confidence=0.5):
        print ("FaceDetector -> init")
        self.net = cv.dnn.readNetFromCaffe(configFile, modelFile)
        self.min_confidence = min_confidence
        print ("FaceDetector -> init ok")
    
    def detect(self, image):
        blob = cv.dnn.blobFromImage(image, 1.0, (100, 100), [106, 121, 150], False, False)
        frameHeight, frameWidth, channels = image.shape
        self.net.setInput(blob)
        detections = self.net.forward()
        faces_result=[]
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > self.min_confidence:
                x1 = int(detections[0, 0, i, 3] * frameWidth)
                y1 = int(detections[0, 0, i, 4] * frameHeight)
                x2 = int(detections[0, 0, i, 5] * frameWidth)
                y2 = int(detections[0, 0, i, 6] * frameHeight)
                f = (x1,y1, x2-x1, y2-y1)
                if f[2]>1 and f[3]>1:
                    faces_result.append({
                        'roi': f,
                        'type': 'face',
                        'img': image[f[1]:f[1]+f[3], f[0]:f[0]+f[2]],
                        'confidence' : confidence
                    })
        return faces_result
    
    def __del__(self):
        print ("FaceDetector -> bye")


# **PREPROCESSING IMMAGINI**

In [9]:
from PIL import Image
from IPython.display import display
import dlib
import cv2 as cv
from random import getrandbits,randint
from imutils import rotate



size = (100,100,3)
  

def preprocessing(img,resize=False,size=None):
  """Effettua il preprocessing dell'immagine

  Parameters
  ----------
  img : np.ndarray
      L'immagine da preprocessare
  label: int
      Label associata all'immagine
  resize: bool, optional
      Se true effettua il resize dell'immagine 
  size: tuple, optional
      Se resize = True, effettua il resize della dimensione specificata

  Returns
  -------
  tf.Tensor
      L'immagine preprocessata sotto forma di tensore
  """
  if resize:
    img = cv.resize(img,(size,size))
  
  #applica il clahe
  lab = cv.cvtColor(img, cv.COLOR_BGR2LAB)
  lab_planes = cv.split(lab)
  lab_planes[0] = clahe.apply(lab_planes[0])
  lab = cv.merge(lab_planes)
  img = cv.cvtColor(lab, cv.COLOR_LAB2BGR)


  img = img.astype(np.float32)
  img = img / 255.0
 
  return img



def test_preprocess(sample):
  """
     Effettua il preprocessing sull'immagine. In fase di test non viene effettuata data augmentation

  Parameters
  ----------
  sample : dict
        Dizionario contente un campione del dataset. Contiene il nome del file, l'immagine e la label

  Returns
  tuple
    Tupla che contiene il nome del file e l'immagine processata
  """
  image = tf.py_function(detect_and_align,[sample['image'],size],tf.float32)
  return sample['file_name'],image


def findRelevantFace(objs, W,H):
  """Trova il volto principale tra i volti trovati nella face detection
  Parameters
  ----------
  objs : tf.Tensor
        lista di dizionari, ogni elemento della lista contiene informazioni su un volto rilevato
  W : tf.Tensor
        larghezza dell'immagine originale
  H : int
        lunghezza dell'immagine originale
  Returns
  bool
    True se label != 255, False altrimenti

  """
  mindistcenter = None
  minobj = None
  for o in objs:
      cx = o['roi'][0] + (o['roi'][2]/2)
      cy = o['roi'][1] + (o['roi'][3]/2)
      distcenter = (cx-(W/2))**2 + (cy-(H/2))**2
      if mindistcenter is None or distcenter < mindistcenter:
          mindistcenter = distcenter
          minobj = o
  return minobj


def detect_and_align(img,size):
  """Effettua la detection della faccia nell'immagine e allinea il volto.

  Parameters
  ----------
  img : tf.Tensor
        Tensore che contiene l'immagine
  label : tf.Tensor
        Tensore che contine la label
  size : tuple
        Dimensione desiderata dell'immagine preprocessata
  Returns
  Tf.Tensor
    Tensore che contine l'immagine preprocessata
  """
  resize = True
  img_array = img.numpy()
  #img_array = img.copy()
  det_imgs = detector.detect(img_array)
  if len(det_imgs) > 0:
    rel = findRelevantFace(det_imgs,img_array.shape[0],img_array.shape[1])
    roi = rel['roi']
    rect = dlib.rectangle(roi[0],roi[1],roi[0]+roi[2],roi[1]+roi[3])
    face = sp(img_array, rect)
    img_array = dlib.get_face_chip(img_array, face, size=size[0])

  return preprocessing(img_array,True,size[0])

  

detector=FaceDetector()
clahe = cv.createCLAHE(2,(3,3))
predictor_path = "/home/paolo/ComputerSync/Università/ProgettoAgeEstimation/data/shape_predictor_68_face_landmarks.dat"
sp = dlib.shape_predictor(predictor_path)
net = cv.dnn.readNetFromCaffe(configFile, modelFile)
size = (100,100,3)




FaceDetector -> init
FaceDetector -> init ok


In [4]:
import tensorflow.keras.backend as K

def mae(y_true,y_pred):
  return K.mean(K.abs(K.argmax(y_true,-1)-K.argmax(y_pred,-1)),0)


In [5]:
tf.compat.v1.reset_default_graph()
tf.keras.backend.clear_session()
model = tf.keras.models.load_model('/home/paolo/ComputerSync/Università/ProgettoAgeEstimation/AgeEstimation/model/model.h5',
                                compile=False)

In [10]:
import csv
AUTOTUNE = tf.data.experimental.AUTOTUNE


#carica il dataset
ds,ds_info = tfds.load(
    name='vgg_face2',
    split='test',
    data_dir='/home/paolo/ComputerSync/Università/ProgettoAgeEstimation/AgeEstimation/vggface2',
    shuffle_files = False,
    as_supervised = False,
    with_info=True
)

test_ds = ds.map(test_preprocess,num_parallel_calls=1)
test_ds = test_ds.prefetch(AUTOTUNE)


#effettua la prediction sul test set
predictions=[]
for el in test_ds:
    filename = el[0].numpy().decode('UTF-8')
    prediction = model.predict(el[1].numpy().reshape(1,100,100,3))
    pred = np.argmax(prediction) + 1 
    print(filename,pred)
    predictions.append([filename,pred])

    

with open('GROUP_xx.csv', 'w') as f:       
    write = csv.writer(f) 
    write.writerows(predictions) 




jpg 58
n004007/0133_01.jpg 56
n008937/0107_01.jpg 32
n008164/0039_01.jpg 37
n004239/0042_01.jpg 42
n000029/0308_01.jpg 60
n007909/0347_02.jpg 13
n002581/0141_02.jpg 38
n009232/0410_01.jpg 49
n003140/0237_02.jpg 26
n007703/0264_03.jpg 32
n001830/0433_01.jpg 55
n004243/0344_01.jpg 54
n004469/0014_01.jpg 28
n005427/0080_04.jpg 33
n007865/0092_03.jpg 37
n006992/0049_03.jpg 40
n004243/0121_01.jpg 53
n004208/0123_01.jpg 40
n000596/0262_01.jpg 72
n002166/0025_02.jpg 42
n005145/0118_04.jpg 49
n001368/0026_02.jpg 52
n003653/0256_02.jpg 55
n001817/0124_01.jpg 47
n007900/0400_02.jpg 49
n006922/0175_01.jpg 52
n004850/0007_01.jpg 32
n001127/0072_01.jpg 35
n006247/0328_02.jpg 34
n004366/0411_01.jpg 61
n004743/0440_02.jpg 24
n005101/0041_02.jpg 40
n007919/0237_01.jpg 25
n006772/0630_01.jpg 18
n004007/0054_04.jpg 58
n004586/0554_01.jpg 32
n005225/0391_04.jpg 37
n001485/0411_01.jpg 43
n005709/0190_01.jpg 49
n005181/0114_02.jpg 42
n009123/0179_01.jpg 26
n007296/0091_01.jpg 67
n006247/0241_01.jpg 32
n005